# BERTHA LUTZ DEV AGENTIC

- LLM responde usando RAG dos protocolos do Ministério da Saúde > Guarda histórico no PostgreSQL > Aplica guardrails (não diagnostica, não prescreve)> Usa LangGraph + tools

## Imports

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

c:\Users\erico\Documents\Bertha-Lutz-AI\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Ingerir PDFs do MS no Chroma (RAG)

In [2]:
# rag/ingest.py

def ingest(pdf_path: str):
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    chunks = splitter.split_documents(docs)

    embeddings = HuggingFaceEmbeddings(model_name="hkunlp/instructor-base")

    vectordb = Chroma.from_documents(
        chunks,
        embedding=embeddings,
        persist_directory="chroma_db"
    )

    vectordb.persist()

In [4]:
ingest(r'pdf\Consensointegra.pdf')

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 331.85it/s, Materializing param=shared.weight]                                                     
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
C:\Users\erico\AppData\Local\Temp\ipykernel_18816\3887741603.py:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [5]:
ingest(r'pdf\relatorio-preliminar-diretrizes-brasileiras-para-o-rastreamento-do-cancer-do-colo-do-utero-parte-i-rastreamento-organizado-utilizando-testes-moleculares-para-deteccao-de-dna-hpv-oncogenico.pdf')

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 421.94it/s, Materializing param=shared.weight]                                                     
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [6]:
ingest(r'pdf\Manual da Gestante.pdf')

Loading weights: 100%|██████████| 100/100 [00:00<00:00, 338.49it/s, Materializing param=shared.weight]                                                     
The tied weights mapping and config for this model specifies to tie shared.weight to encoder.embed_tokens.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
